In [13]:
import ipanema
import numpy as np
ipanema.initialize('cuda',1)
from scipy import special
import matplotlib.pyplot as plt
%matplotlib widget
import ipywidgets as widgets
import uproot3 as uproot

Already set "cuda" as (cuda)


In [14]:
branches = ['Bu_M', 'Bu_PVConst_veloMatch', 'Bu_PVConst_veloMatch_stdmethod']
rd = uproot.open("/scratch46/marcos.romero/Bu2JpsiKplus.root")['Bu2JpsiKplus']
rd = rd['DecayTree'].pandas.df(branches=branches)
rd = ipanema.Sample.from_pandas(rd)
mass = rd['Bu_M']

OSError: [Errno 24] Too many open files: '/scratch46/marcos.romero/Bu2JpsiKplus.root'

In [6]:
prog = THREAD.compile(
"""
#define USE_DOUBLE 1
#include <ipanema/core.c>
#include <ipanema/complex.c>
#include <ipanema/special.c>
#include <ipanema/lineshapes.c>
#include <exposed/kernels.ocl>
""", compiler_options=[f"-I{ipanema.IPANEMALIB}"])

def ipatia(x, mu, sigma=1, lambd=-1, zeta=0, beta=-0.01, aL=50, nL=1.5, aR=1., nR=0.1):
  xd = ipanema.ristra.allocate(x)
  yd = ipanema.ristra.allocate(0*x)
  prog.py_ipatia(yd, xd, np.float64(mu), np.float64(sigma),
                  np.float64(lambd), np.float64(zeta), np.float64(beta),
                  np.float64(aL), np.float64(nL),
                  np.float64(aR), np.float64(nR),
                  global_size=(len(x)))
  return ipanema.ristra.get(yd)

In [10]:
@widgets.interact(mu=(0, 10, 1), sigma=(0.0001, 5, 1), lambd=(-10, 10, 1), 
                  zeta=(0, 10, 1), beta=(-5,5,1), 
                  aL=(0,1000,1), nL=(0,10,1), aR=(0,1000,1), nR=(0,10,1))
def show_ipatia(mu=0, sigma=1, lambd=-1, zeta=0, beta=-0.01, aL=50, nL=1.5, aR=1., nR=0.1):
  x = np.linspace(-10,10,100).astype(np.float64)
  fig, ax = ipanema.plotting.axes_plot()
  ip = ipatia(x, 0., 1., -1.0, 0.0, -0.01, 50.0, 1.5, 1.0, 0.1)
  y = ip/np.trapz(ip,x)
  ax.plot(x, y, label=rf"base")    
  ip = ipatia(x, mu, sigma, lambd, zeta, beta, aL, nL, aR, nR)
  y = ip/np.trapz(ip,x)
  ax.plot(x, y, label=rf"$\lambda = {lambd}$")
  #ax.legend()

interactive(children=(IntSlider(value=0, description='mu', max=10), FloatSlider(value=1.0, description='sigma'…